In [1]:
import pandas
import numpy
import os
import math
import warnings
from IPython.core.display import display, HTML

## 전체 구간을 넓게
display(HTML("<style>.container { width:100% !important; }</style>"))
## 각 컬럼 width 최대로
pandas.set_option('display.max_colwidth', -1)
## rows 500
pandas.set_option('display.max_rows', 500)
## columns
pandas.set_option('display.max_columns', 500)
pandas.set_option('display.width', 1000)

pandas.options.display.float_format = '{:.5f}'.format
warnings.filterwarnings(action = 'ignore')

PATH = 'C:/Users/WAI/OneDrive/04.DOMHWANGCHA/DOMHWANGCHA/99.DATA/98.DACON/01.주차수요예측AI경진대회/'

C:\Users\WAI\Anaconda3\envs\analysis\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  # This is added back by InteractiveShellApp.init_path()


In [2]:
df = pandas.read_csv(PATH + "train.csv", index_col = False)
test_df = pandas.read_csv(PATH + "test.csv", index_col = False)
df = df.append(test_df)
age_df = pandas.read_csv(PATH + "age_gender_info.csv", index_col=False)

print("Training Set Shape : {}".format(df.shape))
print("Test Set Shape : {}".format(test_df.shape))
print("Total Set Shape : {}".format(df.shape))
print("Age Gender Set Shape : {}".format(age_df.shape))


Training Set Shape : (2952, 15)
Test Set Shape : (1022, 14)
Total Set Shape : (3974, 15)
Age Gender Set Shape : (16, 23)


In [13]:
print("전체 아파트 단지 : {}".format(len(df["단지코드"].unique())))
print("공급유형 : {}\n".format(df["공급유형"].value_counts()))
print("임대건물구분: {}\n".format(df["임대건물구분"].value_counts()))
#df.groupby(["단지코드", "임대건물구분"]).count() # Pivot으로 아파트 ,상가 단지 구분


전체 아파트 단지 : 573
공급유형 : 국민임대         2380
임대상가         739 
행복주택         337 
공공임대(10년)    240 
영구임대         197 
공공임대(50년)    44  
공공임대(분납)     18  
장기전세         9   
공공분양         7   
공공임대(5년)     3   
Name: 공급유형, dtype: int64

임대건물구분: 아파트    3235
상가     739 
Name: 임대건물구분, dtype: int64



In [24]:
# 이상한 관측치 철
df = df.replace("-",numpy.nan)
df = df.astype({"임대보증금" : float, "임대료" : float}) 
# null 값 확인
df = df.rename(columns = {"도보 10분거리 내 지하철역 수(환승노선 수 반영)" : "인접지하철역수", "도보 10분거리 내 버스정류장 수": "인접버스정거장수"})
df.describe().T

,count,mean,std,min,25%,50%,75%,max
총세대수,3974.00000,880.33971,519.54239,26.00000,504.00000,775.00000,1116.00000,2572.00000
전용면적,3974.00000,44.48695,32.95255,9.96000,32.12750,39.84000,51.28250,583.40000
전용면적별세대수,3974.00000,102.14796,130.95242,1.00000,14.00000,60.00000,142.00000,1865.00000
공가수,3974.00000,13.59562,10.91349,0.00000,4.00000,13.00000,21.00000,55.00000
임대보증금,3207.00000,26391607.17181,20396806.18297,2249000.00000,14487000.00000,21015000.00000,32443000.00000,216423000.00000
임대료,3198.00000,190975.06567,123166.23443,16650.00000,110877.50000,159960.00000,232200.00000,1058030.00000
인접지하철역수,3721.00000,0.16608,0.42985,0.00000,0.00000,0.00000,0.00000,3.00000
인접버스정거장수,3970.00000,3.93526,3.59174,0.00000,2.00000,3.00000,4.00000,50.00000
단지내주차면수,3974.00000,588.06467,383.95503,13.00000,280.00000,499.50000,804.00000,1798.00000
등록차량수,2952.00000,559.76829,433.37503,13.00000,220.00000,487.00000,770.00000,2550.00000


In [25]:
df.isnull().sum().to_frame().T

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,인접지하철역수,인접버스정거장수,단지내주차면수,등록차량수,전용면적평수,평수범주
0,0,0,0,0,0,0,0,0,2,767,776,253,4,0,1022,0,0


In [22]:

# 전용 면적의 그룹핑필요
# 전용 면적 평수 변환하여 평수별 그룹핑

df["전용면적평수"] = [math.ceil(x)/10 for x in df["전용면적"]/3.3*10]
# df["전용면적평수"].hist()
# 임대 상가의 경우 면적평수가 매우 넓음!
# df[df["전용면적평수"] > 60]




# 전용평수 범주화
# 아파트 -> 25.8평이 최대 
print("최대 평수 : {}, 최소 평수 : {}, 평균평수 : {} ".format(df["전용면적평수"].max(),df["전용면적평수"].min(),round(df["전용면적평수"].mean(),1)))
# 범주 구분 
# 원룸 -> 10평 미만 -> 1001
# 10평이상 13평 미만 -> 2001
# 13평이상 18평 미만 -> 3001
# 18평이상 21평 미만 -> 4001
# 21평 이상 24평 미만 -> 5001
# 24평 이상 -> 6001

df.loc[df["전용면적평수"] < 10, "평수범주"] = 1001
df.loc[(df["전용면적평수"] >= 10) & (df["전용면적평수"] < 13), "평수범주"] = 2001
df.loc[(df["전용면적평수"] >= 13) & (df["전용면적평수"] < 15), "평수범주"] = 3001
df.loc[(df["전용면적평수"] >= 15) & (df["전용면적평수"] < 18), "평수범주"] = 4001
df.loc[(df["전용면적평수"] >= 18) & (df["전용면적평수"] < 21), "평수범주"] = 5001
df.loc[(df["전용면적평수"] >= 21) & (df["전용면적평수"] < 24), "평수범주"] = 6001
df.loc[(df["전용면적평수"] >= 24) & (df["전용면적평수"] < 28), "평수범주"] = 7001
df.loc[(df["전용면적평수"] >= 28), "평수범주"] = 8001


df = df.astype({"평수범주" : numpy.int16})



아파트 최대 평수 : 176.8, 최소 평수 : 3.1, 평균평수 : 13.5 


In [38]:
# 아파트와 상가로 데이터 분할
# Null 포함되지 않은 데이터셋과, Null 포함된 데이터 분할 
df_apt = df.loc[df["임대건물구분"] == "아파트"]
df_store = df.loc[df["임대건물구분"] == "상가"]

print("아파트 : {} , 상가 : {}".format(len(df_apt), len(df_store)))

df_apt_null = df_apt[df_apt["임대보증금"].isnull() == True]
df_store_null = df_store[df_store["임대보증금"].isnull() == True]

df_apt_not_null = df_apt[df_apt["임대보증금"].isnull() != True]
df_store_not_null = df_store[df_store["임대보증금"].isnull() != True]




아파트 : 3235 , 상가 : 739


In [54]:
# 1. 임대보증금
# 1. 임대보증금, 임대료가 NaN인 데이터에서 상가 아파트에 따른 정보 확인 
print("임대보증금이 임대건물구분별 Null 데이터 : \n {}".format(df[df["임대보증금"].isnull() == True].groupby(["임대건물구분"]).count()["단지코드"]))
print("---------------------------------------------------------")
print("임대보증금이 자격유형별 Null 데이터 : \n {}".format(df[df["임대보증금"].isnull() == True].groupby(["자격유형"]).count()["단지코드"]))
print("---------------------------------------------------------")
print("결측이 존재하는 아파트 건수 : {}".format(len(df_apt_null)))
print("---------------------------------------------------------")
print("결측이 존재하는 아파트 건수 : {}".format(len(df_store_null)))
print("---------------------------------------------------------")
print("Null이 존재하는 Apt의 단지코드 : {}".format(df_apt_null["단지코드"].unique()))
# ['C1350' 'C1326' 'C1786' 'C2186' 'C1006' 'C2152' 'C1267']

df.loc[df["단지코드"].isin(['C1350', 'C1326', 'C1786', 'C2186', 'C1006', 'C2152', 'C1267'])]

df_apt_null.loc[df_apt_null["단지코드"].isin(['C1350', 'C1326', 'C1786', 'C2186', 'C1006', 'C2152', 'C1267'])][["지역","공급유형","평수범주","임대보증금","임대료"]]

print("결측치가 존재하는 아파트 지역 : {}".format(df.loc[df["단지코드"].isin(['C1350', 'C1326', 'C1786', 'C2186', 'C1006', 'C2152', 'C1267'])]["지역"].unique()))
# ['대전광역시' '부산광역시' '강원도' '대구광역시' '경상남도']

임대보증금이 임대건물구분별 Null 데이터 : 
 임대건물구분
상가     739
아파트    28 
Name: 단지코드, dtype: int64
---------------------------------------------------------
임대보증금이 자격유형별 Null 데이터 : 
 자격유형
C    2  
D    749
H    8  
K    4  
L    4  
Name: 단지코드, dtype: int64
---------------------------------------------------------
결측이 존재하는 아파트 건수 : 28
---------------------------------------------------------
결측이 존재하는 아파트 건수 : 739
---------------------------------------------------------
Null이 존재하는 Apt의 단지코드 : ['C1350' 'C1326' 'C1786' 'C2186' 'C1006' 'C2152' 'C1267']
결측치가 존재하는 아파트 지역 : ['대전광역시' '부산광역시' '강원도' '대구광역시' '경상남도']


In [66]:
# 임대보증금 보완을 위한 데이터 처리 
df_area = df_apt_not_null.loc[df_apt_not_null["지역"].isin(['대전광역시', '부산광역시', '강원도', '대구광역시', '경상남도'])][["지역","공급유형","평수범주","임대보증금","임대료"]]
df_grouped_area = df_area.groupby(["지역","공급유형","평수범주"]).mean()

df_grouped_area.to_csv(PATH + "grouped_area.csv", index = True,encoding = "utf-8")

In [64]:
# Null 지역의 공급유형 특징 파악

df_apt_null[["지역","공급유형","평수범주","임대보증금","임대료"]].groupby(["지역","공급유형","평수범주"]).max()
df_grouped_null_area = df_apt_null[["지역","공급유형","전용면적","평수범주","임대보증금","임대료"]]
df_grouped_null_area.to_csv(PATH + "grouped_null_area.csv", index=True, encoding = "utf-8")

# 범위에 없는 결측치도 발생..
# Test + Train 합치고
# 에러 데이터 삭제후 진행 